In [ ]:
# pip install --upgrade praw
# pip install nltk
# pip install openai
# nltk.download('vader_lexicon')

### 1. Scraping the posts and comments from Reddit about clinical trials

In [90]:
import praw

In [91]:
reddit = praw.Reddit(
    client_id = "",
    client_secret = "",
    password = "",
    user_agent = "",
    username = ""
)

subreddit = reddit.subreddit('all') # accessing all of reddit

In [92]:
upvotes_post_list = []

for i in subreddit.search("clinical trials", limit = 2): # searching for posts related to 'clinical trials' and limiting to 10 posts.
    upvotes_post_list.append([i.score, i])

upvotes_post_list.sort(key= lambda x:x[0], reverse=True) # ordering in descending order by the number of upvotes


In [93]:
def extract_comment_tree(post_text, comments, array): # extract the comment trees from the post

    for comment in comments:
        
        if comment.author is not None and comment.author.name!="AutoModerator":
            comment_text = comment.author.name + "[text part]" + " Referring text: " + post_text + " Comment: " + comment.body
            array.append(comment_text)

            replies = [x for x in comment.replies][:-1]

            if replies == []:
                continue
            else:
                extract_comment_tree(comment.body, replies, array)
        else:
            continue
        
    return array

In [94]:
data = {}

for post in upvotes_post_list:
    post_text = post[1].author.name + "[text part]" + post[1].title + " " + post[1].selftext
    data[post_text] = []
    comments = [x for x in post[1].comments][:-1]

    if comments != []:
        data[post_text] = extract_comment_tree(post_text.split("[text part]")[1], comments, data[post_text])

### 2. Analyzing Sentiment of the post/comment and Generating personalized reply to motivate participation in the clinical trial

In [23]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from openai import OpenAI
import time

In [ ]:
# asking chatgpt for the posts
def ask_chatgpt_post(author_name, text, score, prompts): 
    api_key = ''

    client = OpenAI(
        api_key = api_key   #the API key
    )
    messages = [ {"role": "system", "content": "You are a intelligent assistant."} ]
    
    if score < -0.5:
        prompt = prompts["Very Negative"]
    elif score >= -0.5 and score < -0.05:
        prompt = prompts["Negative"]
    elif score >= -0.05 and score <= 0.05:
        prompt = prompts["Neutral"]
    elif score > 0.05 and score <= 0.5:
        prompt = prompts["Positive"]
    else:
        prompt = prompts["Very Positive"]

    message =f"This is the view of a user (name: {author_name}) on a particular clinical trial: {text} The vader sentiment scores for this view is {str(score)}. {prompt}"

    messages.append(
        {"role": "user", "content": message},
    )

    if message:
        chat_completion = client.chat.completions.create(
            messages=messages,
            model="gpt-3.5-turbo-0125",
        )

        reply = chat_completion.choices[0].message.content  
    else:
        reply = "Joining a clinical trial not only offers potential medical breakthroughs but empowers you to contribute to cutting-edge research shaping the future of healthcare—your participation matters."

    return reply
    
# asking chatgpt for the comments
def ask_chatgpt_comment(author_name, text, post_text, score, prompts): 
    api_key = ''

    client = OpenAI(
        api_key = api_key   #the API key
    )
    messages = [ {"role": "system", "content": "You are a intelligent assistant."} ]
    
    if score < -0.5:
        prompt = prompts["Very Negative"]
    elif score >= -0.5 and score < -0.05:
        prompt = prompts["Negative"]
    elif score >= -0.05 and score <= 0.05:
        prompt = prompts["Neutral"]
    elif score > 0.05 and score <= 0.5:
        prompt = prompts["Positive"]
    else:
        prompt = prompts["Very Positive"]

    message =f"This is the comment of a user (name: {author_name}) on a particular clinical trial: {text}. The comment is on the following post: {post_text}. The vader sentiment scores for this comment is {str(score)}. {prompt}"

    messages.append(
        {"role": "user", "content": message},
    )
    
    if message:
        chat_completion = client.chat.completions.create(
            messages=messages,
            model="gpt-3.5-turbo-0125",
        )

        reply = chat_completion.choices[0].message.content  
    else:
        reply = "Joining a clinical trial not only offers potential medical breakthroughs but empowers you to contribute to cutting-edge research shaping the future of healthcare—your participation matters."

    return reply

In [ ]:
# vader sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# a dictionary with "post and the corresponding reply message" as the keys and "comment and corresponding reply message" as the values.
replies = {}

# prompt according to the sentiment score
prompts = {
    "Very Negative": "Write a personalized message for this user convincing them to consider the clinical trial, addressing their concerns.",
    "Negative": "Craft a message addressing concerns and providing reassurance about the clinical trial.",
    "Neutral": "Generate a message encouraging consideration of the clinical trial and providing general information.",
    "Positive": "Create a message highlighting the benefits of the clinical trial and encouraging exploration of available options.",
    "Very Positive": "Compose an enthusiastic message promoting the benefits of the clinical trial and encouraging active participation."
}


for post in data.keys():
    post_author_name = post.split("[text part]")[0]
    post_text = post.split("[text part]")[1]
    post_score = analyzer.polarity_scores(post_text)

    post_reply = ask_chatgpt_post(post_author_name, post_text, post_score, prompts)
    
    post_reply = post_reply.replace("[Your Name]", "Turmerik Team")

    replies[post + " Reply Message: " + post_reply] = []

    for comment in data[post]:
        author_name = comment.split("[text part]")[0]
        text = comment.split("[text part]")[1]
        score = analyzer.polarity_scores(text)

        reply = ask_chatgpt_comment(author_name, text, post_text, score, prompts)

        reply = reply.replace("[Your Name]", "Turmerik Team")

        replies[post + " Reply message: " + post_reply].append(author_name + " Comment: " + text.split("Comment:")[1] + " Reply Message: " + reply)
    